In [1]:
# Data stuff
import numpy as np
import pandas as pd

# Visual stuff
from IPython.display import display
# Configs 
pd.options.display.max_columns = None
pd.options.display.max_rows = None
import matplotlib.pyplot as plt

# Random state seed
rand_state=42

# ML stuff
import sklearn
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV,StratifiedKFold
import lightgbm as lgb
import xgboost as xgb
from xgboost import XGBClassifier
import shap

In [2]:
dataset = pd.read_csv('data_gen/dataset_missing_undersampled.csv')
# Ensure correct types
dataset = dataset.astype(dtype={
    'age':float,
    'gender':int,
    'expired':int,
    'P-glucose':float,
    'blood_pressure_systoliskt':float,
    'blood_pressure_diastoliskt':float,
    'BMI':float
    
})
# Order ints(categorical variables, first) and floats(number variables, last)
dataset = dataset[[
    'gender',
    'I109',
    'E119',
    'E669',
    'I259',
    'I252',
    'I209',
    'E660',
    'E118',
    'I639',
    'E113',
    'expired',
    'age',
    'P-glucose',
    'blood_pressure_systoliskt',
    'blood_pressure_diastoliskt',
    'BMI',
]]

In [3]:
# Generate data set withouth categories(all numbers)
Y_no_cat = dataset.expired.values
X_no_cat = dataset.drop(columns=['expired']).values

# dataset as is, but target variable dropped(hospital expire flag)
dataset_no_target = dataset.drop(columns=['expired'])

# Feture names and categorical feature names
feature_names = dataset_no_target.select_dtypes(include='int').columns.values.tolist() + dataset_no_target.select_dtypes(exclude='int').columns.values.tolist() 
cat_feature_names = dataset_no_target.select_dtypes(include='int').columns.values.tolist() 

# Generate data set with categories(int type required)
dataframe_int_list = dataset_no_target.select_dtypes(include='int').values.tolist()
dataframe_no_int_list = dataset_no_target.select_dtypes(exclude='int').values.tolist()
Y = dataset.expired.values.tolist()
X = []
for i,v in enumerate(dataframe_int_list):
    X = X + [v+dataframe_no_int_list[i]]

# Generate categorical feature indicies
cat_features_indices=list(range(0,len(dataframe_int_list[0])))

In [4]:
# Function that w
def strat_cv_it(classifier, params, uses_cat, param_comb ):
    folds = 3
    skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = rand_state)
    
    # Perform fit and scoring
    if uses_cat:
        random_search = RandomizedSearchCV( classifier, return_train_score=True, param_distributions=params ,scoring='roc_auc', n_iter=param_comb, n_jobs=4, cv=skf.split(X, Y),random_state = rand_state, refit=True, verbose=3, error_score=0.0 )
        random_search.fit(X, Y)     
    else:
        random_search = RandomizedSearchCV( classifier, return_train_score=True, param_distributions=params ,scoring='roc_auc', n_iter=param_comb, n_jobs=4, cv=skf.split(X_no_cat, Y_no_cat),random_state = rand_state, refit=True, verbose=3,error_score=0.0)
        random_search.fit(X_no_cat, Y_no_cat)
    
    # Display results and return best model
    display(random_search.best_score_)
    display(random_search.best_params_)
    display(pd.DataFrame(random_search.cv_results_))
    return random_search.best_estimator_

# Scale of negative class to the positive class(#survived/#died)
scale_pos_weight_min = int( (dataset[dataset.expired==0].shape[0] / dataset[dataset.expired==1].shape[0]) )

# XGboost

In [5]:
xgb_classifier = xgb.XGBClassifier(objective = "binary:logistic",random_state=rand_state)
xgb_params = {
        'learning_rate': (0.01, 0.05,0.1),
        'min_child_weight': [3, 5, 10],
        'gamma': [0.5, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': list(range(5,30)),
      #  'scale_pos_weight':  [0.5,0.75,0.9,1, 1.1, 1.25,1.5]
}
model = strat_cv_it(xgb_classifier ,xgb_params,False,50)
explainer = shap.TreeExplainer(model)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:  1.0min
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:  4.1min
[Parallel(n_jobs=4)]: Done 150 out of 150 | elapsed:  4.5min finished


0.8797374588108559

{'subsample': 0.8,
 'min_child_weight': 10,
 'max_depth': 24,
 'learning_rate': 0.01,
 'gamma': 5,
 'colsample_bytree': 1.0}

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample,param_min_child_weight,param_max_depth,param_learning_rate,param_gamma,param_colsample_bytree,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,8.338674,5.581379,0.039483,0.007421,0.8,10,12,0.1,2,1,"{'subsample': 0.8, 'min_child_weight': 10, 'ma...",0.860853,0.851501,0.889067,0.867140,0.015968,32,0.926542,0.917968,0.913034,0.919181,0.005581
1,9.550496,4.430831,0.040984,0.013984,0.8,10,24,0.01,5,1,"{'subsample': 0.8, 'min_child_weight': 10, 'ma...",0.879665,0.861369,0.898178,0.879737,0.015027,1,0.898792,0.899881,0.891513,0.896729,0.003715
2,11.580574,6.561259,0.039496,0.011282,1,5,25,0.01,1.5,0.6,"{'subsample': 1.0, 'min_child_weight': 5, 'max...",0.864358,0.842664,0.897940,0.868321,0.022739,24,0.926262,0.915876,0.911680,0.917939,0.006129
3,8.415505,3.413740,0.041811,0.003825,1,10,28,0.1,5,0.8,"{'subsample': 1.0, 'min_child_weight': 10, 'ma...",0.872592,0.840170,0.898351,0.870371,0.023804,17,0.908186,0.905188,0.894367,0.902580,0.005935
4,7.604951,1.421723,0.049241,0.015514,0.6,10,10,0.1,5,0.8,"{'subsample': 0.6, 'min_child_weight': 10, 'ma...",0.872442,0.848147,0.894434,0.871674,0.018904,11,0.907318,0.903198,0.891744,0.900753,0.006589
5,5.593754,2.940900,0.036935,0.000944,0.8,10,6,0.1,5,0.8,"{'subsample': 0.8, 'min_child_weight': 10, 'ma...",0.872872,0.845309,0.895343,0.871174,0.020462,14,0.911653,0.905587,0.895523,0.904254,0.006652
6,10.731557,3.327101,0.034065,0.008352,0.8,3,24,0.05,1.5,0.8,"{'subsample': 0.8, 'min_child_weight': 3, 'max...",0.856080,0.829076,0.890906,0.858688,0.025309,50,0.956201,0.958261,0.947552,0.954005,0.004640
7,9.949928,1.657986,0.035694,0.013392,1,5,23,0.05,0.5,0.8,"{'subsample': 1.0, 'min_child_weight': 5, 'max...",0.860273,0.839676,0.893006,0.864318,0.021959,42,0.955317,0.954411,0.945568,0.951766,0.004398
8,13.046614,1.652538,0.045901,0.006552,0.8,3,17,0.05,0.5,1,"{'subsample': 0.8, 'min_child_weight': 3, 'max...",0.854296,0.831484,0.891664,0.859148,0.024807,48,0.966118,0.969532,0.959328,0.964993,0.004241
9,13.912465,2.787502,0.050748,0.010052,0.8,3,26,0.01,1.5,1,"{'subsample': 0.8, 'min_child_weight': 3, 'max...",0.861907,0.836515,0.894910,0.864444,0.023907,41,0.935866,0.935801,0.920226,0.930631,0.007358


In [ ]:
shap_values = explainer.shap_values( X_no_cat)
shap.summary_plot(shap_values,X_no_cat ,feature_names=feature_names, show=False)
#plt.savefig("shap_summary.svg", format='svg', dpi=300, bbox_inches='tight')
for predictor in feature_names:
    if predictor != 'age':
        save = shap.dependence_plot(predictor, shap_values,X_no_cat ,feature_names=feature_names,interaction_index='age', show=False)
    else:
        save = shap.dependence_plot(predictor, shap_values,X_no_cat ,feature_names=feature_names, interaction_index='gender', show=False)
        
    # plt.savefig(predictor+".svg", format='svg', dpi=300, bbox_inches='tight')

# LightGBM

In [7]:
LGB_classifier = lgb.LGBMClassifier()
LGB_params = {
             'num_leaves': [1,5,8,10,15,20,35,40], 
             'min_child_samples': [1,5,10,20,50,100,200,300,400,500], 
             'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
             'subsample': [0.2,0,4,0.5, 0.6, 0.8, 1.0],
             'colsample_bytree': [0.6, 0.8, 1.0],
             'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100],
           #  'scale_pos_weight':  [0.5,0.75,0.9,1, 1.1, 1.25,1.5]

}
model = strat_cv_it(LGB_classifier ,LGB_params,True,50)
explainer = shap.TreeExplainer(model)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    4.9s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:   20.8s
[Parallel(n_jobs=4)]: Done 150 out of 150 | elapsed:   23.4s finished


0.873375678564747

{'subsample': 0.2,
 'reg_lambda': 20,
 'reg_alpha': 7,
 'num_leaves': 35,
 'min_child_weight': 1e-05,
 'min_child_samples': 100,
 'colsample_bytree': 0.6}

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample,param_reg_lambda,param_reg_alpha,param_num_leaves,param_min_child_weight,param_min_child_samples,param_colsample_bytree,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,0.142683,0.018978,0.022384,0.014566,0.6,50,100,5,0.1,20,0.6,"{'subsample': 0.6, 'reg_lambda': 50, 'reg_alph...",0.816090,0.806502,0.500000,0.707531,0.146799,19,0.837570,0.855781,0.500000,0.731117,0.163593
1,0.002404,0.000251,0.000000,0.000000,4,100,7,10,1,400,0.8,"{'subsample': 4, 'reg_lambda': 100, 'reg_alpha...",0.000000,0.000000,0.000000,0.000000,0.000000,38,0.000000,0.000000,0.000000,0.000000,0.000000
2,1.329873,0.207876,0.026365,0.007109,0.5,100,10,20,10,20,0.6,"{'subsample': 0.5, 'reg_lambda': 100, 'reg_alp...",0.876548,0.823787,0.895970,0.865435,0.030498,11,0.882404,0.897002,0.877393,0.885600,0.008318
3,0.667782,0.289310,0.027438,0.005979,0.5,0.1,1,8,0.001,300,0.6,"{'subsample': 0.5, 'reg_lambda': 0.1, 'reg_alp...",0.842084,0.793193,0.856951,0.830743,0.027236,17,0.834992,0.856212,0.823375,0.838193,0.013595
4,0.673354,0.266177,0.029661,0.024795,0.6,0,50,8,1e-05,20,0.6,"{'subsample': 0.6, 'reg_lambda': 0, 'reg_alpha...",0.848405,0.816456,0.895689,0.853517,0.032548,13,0.846657,0.883418,0.845144,0.858406,0.017697
5,0.007755,0.006158,0.000000,0.000000,0,20,10,15,0.001,1,1,"{'subsample': 0, 'reg_lambda': 20, 'reg_alpha'...",0.000000,0.000000,0.000000,0.000000,0.000000,38,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.007668,0.001373,0.000000,0.000000,0.6,5,5,1,1000,20,0.6,"{'subsample': 0.6, 'reg_lambda': 5, 'reg_alpha...",0.000000,0.000000,0.000000,0.000000,0.000000,38,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.345509,0.355104,0.024761,0.004515,0.5,0.1,100,40,1,300,1,"{'subsample': 0.5, 'reg_lambda': 0.1, 'reg_alp...",0.500000,0.783604,0.500000,0.594535,0.133692,24,0.500000,0.839080,0.500000,0.613027,0.159844
8,1.527260,0.416711,0.027680,0.007612,0.8,0.1,0,10,0.001,100,1,"{'subsample': 0.8, 'reg_lambda': 0.1, 'reg_alp...",0.855048,0.836494,0.887292,0.859611,0.020988,12,0.920745,0.924186,0.912765,0.919232,0.004784
9,0.340427,0.159464,0.041569,0.008857,0.2,20,7,10,1000,50,0.6,"{'subsample': 0.2, 'reg_lambda': 20, 'reg_alph...",0.500000,0.500000,0.500000,0.500000,0.000000,26,0.500000,0.500000,0.500000,0.500000,0.000000


In [ ]:
shap_values = explainer.shap_values(dataset.drop(columns=['expired']))
shap.summary_plot(shap_values[1],dataset.drop(columns=['expired']) ,feature_names=feature_names, show=False)
#plt.savefig("shap_summary.svg", format='svg', dpi=300, bbox_inches='tight')
for predictor in feature_names:
    if predictor != 'age':
        save = shap.dependence_plot(predictor, shap_values[1],dataset.drop(columns=['expired']) ,feature_names=feature_names,interaction_index='age', show=False)
    else:
        save = shap.dependence_plot(predictor, shap_values[1],dataset.drop(columns=['expired']) ,feature_names=feature_names, interaction_index='gender', show=False)
        
  #  plt.savefig(predictor+".svg", format='svg', dpi=300, bbox_inches='tight')